### Launch the ROS demo

In [ ]:
%%bash --bg
trap 'kill $(jobs -p)' EXIT
xterm -e /bin/bash -l -c "roslaunch demo.launch"

Create the controller and run it periodically in a thread

In [ ]:
import rospy
from threading import Thread
from controller import Controller

rospy.init_node('ns_demo')
c = Controller()

def worker():
    rate = rospy.Rate(50)
    while not rospy.is_shutdown():
        c.pose_control(c.im_server.target)
        rate.sleep()
    
t = Thread(target=worker)
t.start()

Create a list of slider widgets, one for each joint, to chose the default pose

In [ ]:
import ipywidgets
from ipywidgets import FloatSlider, Layout, Button, Box
joint_widgets = [FloatSlider(min = j.min, max = j.max, step = (j.max-j.min) / 100, description = j.name) \
                 for j in c.robot.active_joints]

React to slider (value) changes by adapting the default joint pose

In [ ]:
def on_sent(event):
    widget = event.owner
    c.preferred_joints[c.joint_msg.name.index(widget.description)] = widget.value

for widget in joint_widgets:
    widget.observe(on_sent, 'value')

Collect all widgets (sliders and buttons) in a form and display them

In [ ]:
form = Box(joint_widgets, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='100%'
))
display(form)